In [4]:
import pandas as pd
import folium
from folium import Marker
from folium.plugins import MarkerCluster
import sqlalchemy as dbs
import pymysql
from datetime import datetime


cluster = MarkerCluster()
con = pymysql.connect(host='kingsms.asuscomm.com', port=3309, user='root', password='1!Tjalstjs', db='firebase', charset='utf8') # 한글처리 (charset = 'utf8')
 # STEP 3: Connection 으로부터 Cursor 생성
cur = con.cursor()
# STEP 4: SQL문 실행 및 Fetch
sql = "SELECT substr(gps_time, 1, 25) gps_time, trim(gps_latitude), trim(gps_longitude)  FROM gps_history  limit 10000"
sql = "SELECT substr(gps_time,1, 25) gps_time, trim(gps_latitude), trim(gps_longitude) FROM gps_history where gps_time_hour BETWEEN '2024-05-19 20' and '2024-05-20 23' order by gps_time limit 10000"
cur.execute(sql)

rows = cur.fetchall()

row_cnt = cur.rowcount  

con.close()

df = pd.DataFrame(rows) # 쿼리 결과를 dataframe으로 저장

center = [37.306862,  126.9834126]
map = folium.Map(location=center, zoom_start=15)

lines = df[[1,2]].astype(float).values[:].tolist()
times = df[0].apply(lambda _ : datetime.strptime(_,'%Y-%m-%d %H:%M:%S')).tolist()

print(times[0])
print(times[-1])
total_time = (times[-1]-times[0])
print(total_time)

folium.PolyLine(locations = lines).add_to(map)

over_times = [] #30초 이상 머무른 시간
over_gps = [] # 장소
stay_times = [] #30초 이상 머무른 시간

for i in range(row_cnt-1):
    # print(times[i+1])
    # print(times[i])
    # temp = (times[i+1]-times[i]).seconds/60
    # print(round(temp,2))

    temp = (times[i+1]-times[i]).seconds
    # 몇초 멈춘걸 의미 있게 볼 것 인가?
    if temp > 5:
        over_gps.append(lines[i])
        over_times.append(temp)
        stay_times.append(times[i])
        

for i in range(len(over_gps)):
#     folium.CircleMarkerlocation = over_gps[i], tooltip = str(over_times[i])+"초", radius = 10 ,  color='#FF2D00', fill_color='#F38168').add_to(m)

    cluster.add_child(           
#             Marker(location = over_gps[i], popup = "<pre>" + "라벨" + "<br>" + "</pre>",)).add_to(map)
            Marker(location = over_gps[i], popup = "<pre>" + str(stay_times[i]) + "<br> 머무른 : " + str(over_times[i]) + "<br> </pre>",)).add_to(map)
#     over_times[i]

# print(over_times)
# print(over_gps)

map

2024-05-20 18:00:01
2024-05-20 23:35:09
0 days 05:35:08
